In [ ]:
import os
def is_colab():
    try:
        from google.colab import drive
        return True
    except:
        return False
    
if is_colab():
    %cd /content/
    !rm -rf pytorch_random
    
    !git clone https://github.com/Priyanshu-hawk/pytorch_random.git
    if os.getcwd() != '/content/pytorch_random':
        %cd /content/pytorch_random

%cd Vision_Transformer


In [ ]:
from torchvision import datasets
import os
import json
import torch
from tqdm import tqdm
import numpy as np
import shutil

foodDataset = datasets.Food101("../dataset", download=True)


BASE_FOOD_DS = os.path.join("../dataset/food-101")
print(BASE_FOOD_DS)
BASE_FOOD_IMGS = os.path.join(BASE_FOOD_DS, "images")
BASE_FOOD_META = os.path.join(BASE_FOOD_DS, "meta")
TRAIN_JSON_PATH = os.path.join(BASE_FOOD_META, "train.json")
TEST_JSON_PATH = os.path.join(BASE_FOOD_META, "test.json")
CLASSES_TEXT = os.path.join(BASE_FOOD_META, "classes.txt")

TRAIN_JSON = json.load(open(TRAIN_JSON_PATH))
TEST_JSON = json.load(open(TEST_JSON_PATH))

In [ ]:
CLASS_NAMES = []
with open(CLASSES_TEXT, "r") as f:
    for f_p in f.readlines():
        CLASS_NAMES.append(f_p.strip("\n"))
CLASS_NAMES[:5]

In [ ]:
NEW_DS = "../dataset/food101_torch"

# run once
# creating dataset acording to pytorch format

#format = train/class_name/imgs 
#BASE_FOOD_IMGS,TRAIN_JSON[CLASS_NAMES[rand_Class_name]][random.randint(0,750)]+".jpg"



#train loop
if not os.path.exists(NEW_DS):
    os.makedirs(NEW_DS, exist_ok=True)
    for c_i , _class in enumerate(tqdm(CLASS_NAMES)):
        print(_class)
        class_path = os.path.join(NEW_DS, "train", _class)
        os.makedirs(class_path, exist_ok=True)
        i = 0
        for _img in range(0, len(TRAIN_JSON[CLASS_NAMES[0]])):
            img_path = os.path.join(class_path, str(i)+".jpg")
            # print(os.path.join(BASE_FOOD_IMGS,TRAIN_JSON[CLASS_NAMES[c_i][_img]]+".jpg"))
            shutil.copy(os.path.join(BASE_FOOD_IMGS,TRAIN_JSON[CLASS_NAMES[c_i]][_img]+".jpg"),  img_path)
            i+=1

    #test loop
    for c_i , _class in enumerate(tqdm(CLASS_NAMES)):
        print(_class)
        class_path = os.path.join(NEW_DS, "test", _class)
        os.makedirs(class_path, exist_ok=True)
        i = 0
        for _img in range(0, len(TEST_JSON[CLASS_NAMES[0]])):
            img_path = os.path.join(class_path, str(i)+".jpg")
            # print(os.path.join(BASE_FOOD_IMGS,TRAIN_JSON[CLASS_NAMES[c_i][_img]]+".jpg"))
            shutil.copy(os.path.join(BASE_FOOD_IMGS,TEST_JSON[CLASS_NAMES[c_i]][_img]+".jpg"),  img_path)
            i+=1

In [1]:
import os
image_path = "../dataset/pizza_steak_sushi/data"
# image_path = "../dataset/pizza_steak_sushi/"

train_dir = os.path.join(image_path, "train")
test_dir = os.path.join(image_path, "test")

In [2]:
from torchvision import transforms
from data_setup import createDataloader

train_transform = transforms.Compose(
    [
        transforms.Resize((224, 224)) ,
        transforms.ToTensor()

    ]
)

train_DL, test_DL, _classes = createDataloader(train_dir, test_dir, 64, train_transform, train_transform )

In [3]:
len(train_DL)

36

In [4]:
print("total classes:",len(_classes))

total classes: 3


In [5]:
H = 224
W = 224
C = 3
P = 16
N = int((H * W) / P**2)
N

196

In [6]:
import torch
img = torch.randn(1, C, H, W)

In [7]:
from torchinfo import summary

In [8]:
# compliling eq.1 in a single block of code
import torch
from torch import nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

single_img = next(iter(test_DL))[0][0].unsqueeze(0) # single img of batch 1

# patch embb code
class PatchEmbeddingWithPosEmbb(nn.Module):
    def __init__(self, color_channel, out_num_patch, patch_size):
        super().__init__()
        self.conv_layer = nn.Conv2d(in_channels=color_channel,
                               out_channels=out_num_patch,
                               kernel_size=patch_size,
                               stride=patch_size,
                               padding=0)
        self.flat = nn.Flatten(start_dim=2, end_dim=3)

    # Class Token (B, 1, embedding_token) concat to Patched image (B, no_of_patches, embedding_token) = (B, 1+no_of_patches, embedding_token)
    def class_token(self, patch_embb):
        class_token_layer = nn.Parameter(torch.randn(1, 1, patch_embb.shape[2]), requires_grad=True).to(device=device) # (1, 1, 768)
        class_token_expanded = class_token_layer.expand(patch_embb.shape[0], -1, -1) # expending to the batch
        return torch.cat((class_token_expanded, patch_embb),dim=1)

    # Adding Patched Embb with Class Token - (B, 1+no_of_patches, Emmbed_dim)
    def position_embbeding(self, patch_embb):
        pos_embb_layer = nn.Parameter(torch.randn(patch_embb.shape), requires_grad=True).to(device)
        return pos_embb_layer + patch_embb # (B, 196, 768) + (B, 196, 768)

    def forward(self, x):
        x = self.conv_layer(x) # Patched Imgae - (batch, C, H, W) --> (Batch, no_of_patches, embedding_dim patches)
        x = self.flat(x).permute(0,2,1)
        x = self.class_token(x)
        return self.position_embbeding(x)

pe = PatchEmbeddingWithPosEmbb(3, 768, 16).to(device)
gpu_img = img.to(device)
input_patch_embedding = pe(gpu_img)
print(input_patch_embedding.shape)

torch.Size([1, 197, 768])


In [9]:
summary(pe, input_size=(1, 3, 224, 224), col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                 Input Shape          Output Shape         Param #              Trainable
PatchEmbeddingWithPosEmbb (PatchEmbeddingWithPosEmbb)   [1, 3, 224, 224]     [1, 197, 768]        --                   True
├─Conv2d (conv_layer)                                   [1, 3, 224, 224]     [1, 768, 14, 14]     590,592              True
├─Flatten (flat)                                        [1, 768, 14, 14]     [1, 768, 196]        --                   --
Total params: 590,592
Trainable params: 590,592
Non-trainable params: 0
Total mult-adds (M): 115.76
Input size (MB): 0.60
Forward/backward pass size (MB): 1.20
Params size (MB): 2.36
Estimated Total Size (MB): 4.17

In [10]:
class MultiheadSelfAttention(nn.Module):
    def __init__(self, embb_dim: int, num_heads: int, attn_dropout: 0):
        super().__init__()

        # layernorm
        self.layer_norm = nn.LayerNorm(normalized_shape=embb_dim)

        # multi head, we have (B, NumPatch, EmbedDim) or (B, Seq, Feat)
        self.multiHead = nn.MultiheadAttention(embed_dim=embb_dim, num_heads=num_heads, dropout=attn_dropout, batch_first=True)

    def forward(self, x):
        x = self.layer_norm(x)
        atten_out, _ = self.multiHead(query=x, key=x, value=x,need_weights=False)
        return atten_out


In [11]:
class MultiLayerPerceptron(nn.Module):
    def __init__(self, preceptron: int,embb_dim: int, dropout: float):
        super().__init__()

        # layernorm
        self.layer_norm = nn.LayerNorm(normalized_shape=embb_dim)

        self.mlp_layer = nn.Sequential(
            nn.Linear(in_features=embb_dim, out_features=preceptron),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(in_features=preceptron, out_features=embb_dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        x = self.layer_norm(x)
        return self.mlp_layer(x)

In [12]:
class TransformerEncoderBlock(nn.Module):
    def __init__(self, color_c: int, out_patch_num: int, patch_size: int, heads: int, attn_dropout: float, perceptron: int, mlp_dropout: float ):
        super().__init__()
        self.MSA = MultiheadSelfAttention(embb_dim=out_patch_num, num_heads= heads, attn_dropout=attn_dropout)
        self.MLP = MultiLayerPerceptron(perceptron, embb_dim=out_patch_num, dropout=mlp_dropout)

    def forward(self, x):
        msa_out = self.MSA(x) + x
        mlp_out = self.MLP(msa_out) + msa_out
        return mlp_out

# transformer = TransformerEncoderBlock(3, 768, 16, 12, 0, 3072, 0).to(device)
# transformer(input_patch_embedding).shape

In [13]:
torch_transformer_encoder_layer = nn.TransformerEncoderLayer(d_model=768,
                                                             nhead=12,
                                                             dim_feedforward=3072,
                                                             activation=nn.GELU(),
                                                             device=device,
                                                             batch_first=True,
                                                             norm_first=True,
                                                             dropout=0.1)

torch_transformer_encoder_layer

TransformerEncoderLayer(
  (self_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
  )
  (linear1): Linear(in_features=768, out_features=3072, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (linear2): Linear(in_features=3072, out_features=768, bias=True)
  (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (dropout1): Dropout(p=0.1, inplace=False)
  (dropout2): Dropout(p=0.1, inplace=False)
  (activation): GELU(approximate='none')
)

In [14]:
summary(torch_transformer_encoder_layer,
        input_size=(32, 197, 768),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                            Input Shape          Output Shape         Param #              Trainable
TransformerEncoderLayer (TransformerEncoderLayer)  [32, 197, 768]       [32, 197, 768]       --                   True
├─LayerNorm (norm1)                                [32, 197, 768]       [32, 197, 768]       1,536                True
├─MultiheadAttention (self_attn)                   [32, 197, 768]       [32, 197, 768]       2,362,368            True
├─Dropout (dropout1)                               [32, 197, 768]       [32, 197, 768]       --                   --
├─LayerNorm (norm2)                                [32, 197, 768]       [32, 197, 768]       1,536                True
├─Linear (linear1)                                 [32, 197, 768]       [32, 197, 3072]      2,362,368            True
├─GELU (activation)                                [32, 197, 3072]      [32, 197, 3072]      --                   --
├─Dropout (dropout)                            

In [15]:
class VitArch(nn.Module):
    def __init__(self,
                 img_size: int = 224,
                 in_channel: int=3,
                 batch_size: int = 32,
                 patch_size: int = 16,
                 num_transformer_layer:int = 12,
                 embedding_dim: int = 768,
                 mlp_size: int = 3072,
                 num_heads: int=12,
                 atten_dropout: float=0,
                 mlp_dropout: float=0.1,
                 embedding_dropout=0.1,
                 num_classes=1000
                 ):
        super().__init__()
        self.patch_em = PatchEmbeddingWithPosEmbb(in_channel, embedding_dim, patch_size)
        # nn.TransformerEncoderLayer(d_model=embedding_dim,
        #                                                       nhead=num_heads,
        #                                                       dim_feedforward=3072,
        #                                                       dropout=atten_dropout)
        self.embeding_dropout = nn.Dropout(p=embedding_dropout)
        self.transformer_encoder = nn.Sequential(*[TransformerEncoderBlock(color_c=in_channel, out_patch_num=embedding_dim,
                                                           patch_size=patch_size, heads=num_heads, attn_dropout=atten_dropout,
                                                           perceptron=mlp_size, mlp_dropout=mlp_dropout) for _ in range(num_transformer_layer)])

        self.classifer = nn.Sequential(
            nn.LayerNorm(normalized_shape=embedding_dim),
            nn.Linear(in_features=embedding_dim, out_features=num_classes)
        )

    def forward(self, x):
        x = self.patch_em(x)
        x = self.embeding_dropout(x)
        x = self.transformer_encoder(x)
        # print(x.shape, x[:, 0].shape)
        return self.classifer(x[:, 0])

In [25]:
vit = VitArch(num_classes=len(_classes)).to(device)

In [26]:
summary(vit, input_size=(32, 3, 224, 224), col_names=["input_size", "output_size", "num_params", "trainable"], col_width=20, row_settings=["var_names"])

Layer (type (var_name))                                 Input Shape          Output Shape         Param #              Trainable
VitArch (VitArch)                                       [32, 3, 224, 224]    [32, 101]            --                   True
├─Dropout (embeding_dropout)                            [32, 3, 224, 224]    [32, 3, 224, 224]    --                   --
├─PatchEmbeddingWithPosEmbb (patch_em)                  [32, 3, 224, 224]    [32, 197, 768]       --                   True
│    └─Conv2d (conv_layer)                              [32, 3, 224, 224]    [32, 768, 14, 14]    590,592              True
│    └─Flatten (flat)                                   [32, 768, 14, 14]    [32, 768, 196]       --                   --
├─Dropout (embeding_dropout)                            [32, 197, 768]       [32, 197, 768]       --                   --
├─Sequential (transformer_encoder)                      [32, 197, 768]       [32, 197, 768]       --                   True
│    └─Tr

In [51]:
from utils import trainit

loss_fn = nn.CrossEntropyLoss().to(device)
optim = torch.optim.Adam(vit.parameters(), lr=3e-3, betas=(0.9, 0.999), weight_decay=0.3)

trainit(vit, train_DL, test_DL, loss_fn, optim, 50, True, device)

  0%|          | 0/50 [00:02<?, ?it/s]

KeyboardInterrupt



# Using Pre-Train weights

In [1]:
import os
# image_path = "../dataset/pizza_steak_sushi/data"
image_path = "../dataset/food101_torch"

train_dir = os.path.join(image_path, "train")
test_dir = os.path.join(image_path, "test")

In [2]:
import torchvision, torch
from torch import nn
# 1. Get pretrained weights for ViT-Base
pretrained_vit_weights = torchvision.models.ViT_B_16_Weights.DEFAULT

In [3]:
from data_setup import createDataloader

train_transform = pretrained_vit_weights.transforms()

train_DL, test_DL, _classes = createDataloader(train_dir, test_dir, 32, train_transform, train_transform)

print("total classes:",len(_classes))

total classes: 101


In [4]:


# 2. Setup a ViT model instance with pretrained weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pretrained_vit = torchvision.models.vit_b_16(weights=pretrained_vit_weights).to(device)

# 3. Freeze the base parameters
for parameter in pretrained_vit.parameters():
    parameter.requires_grad = False

# 4. Change the classifier head (set the seeds to ensure same initialization with linear head)
# set_seeds()
pretrained_vit.heads = nn.Linear(in_features=768, out_features=len(_classes)).to(device)
# pretrained_vit # uncomment for model output

In [5]:
from torchinfo import summary

In [6]:
# # Print a summary using torchinfo (uncomment for actual output)
summary(model=pretrained_vit,
        input_size=(32, 3, 224, 224), # (batch_size, color_channels, height, width)
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
VisionTransformer (VisionTransformer)                        [32, 3, 224, 224]    [32, 101]            768                  Partial
├─Conv2d (conv_proj)                                         [32, 3, 224, 224]    [32, 768, 14, 14]    (590,592)            False
├─Encoder (encoder)                                          [32, 197, 768]       [32, 197, 768]       151,296              False
│    └─Dropout (dropout)                                     [32, 197, 768]       [32, 197, 768]       --                   --
│    └─Sequential (layers)                                   [32, 197, 768]       [32, 197, 768]       --                   False
│    │    └─EncoderBlock (encoder_layer_0)                   [32, 197, 768]       [32, 197, 768]       (7,087,872)          False
│    │    └─EncoderBlock (encoder_layer_1)                   [32, 197, 768]       [32, 

In [7]:
from utils import trainit

loss_fn = nn.CrossEntropyLoss().to(device)
optim = torch.optim.Adam(pretrained_vit.parameters(), lr=1e-3)

trainit(pretrained_vit, train_DL, test_DL, loss_fn, optim, 50, True, device)

  0%|          | 0/50 [00:00<?, ?it/s]